In [8]:
# 1) 必要ライブラリのインポートと準備
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
from datetime import datetime
import re
from urllib.parse import urljoin
from IPython.display import display
import pandas as pd

print("libraries loaded")

libraries loaded


In [9]:
# 2) SQLite DB 初期化
DB_PATH = "google_repos.sqlite3"

def init_db(path=DB_PATH):
    conn = sqlite3.connect(path)
    cur = conn.cursor()
    cur.execute("""CREATE TABLE IF NOT EXISTS repos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        repo_name TEXT NOT NULL,
        owner TEXT NOT NULL,
        repo TEXT NOT NULL,
        language TEXT,
        stars INTEGER,
        url TEXT,
        scraped_at TEXT
    )""")
    conn.commit()
    return conn

# 初期化して接続確認
conn = init_db()
cur = conn.cursor()
print(f"DB initialized at {DB_PATH}")

DB initialized at google_repos.sqlite3


In [10]:
# Googleリポジトリ一覧（ページ数指定）
BASE_URL = "https://github.com/orgs/google/repositories"
HEADERS = {"User-Agent": "Mozilla/5.0"}

MAX_PAGES = 20  # ← 2000件分スクレイピング

repos = []

for page in range(1, MAX_PAGES + 1):
    url = f"{BASE_URL}?page={page}"
    print(f"[INFO] Fetching page {page} ...")
    time.sleep(1)

    res = requests.get(url, headers=HEADERS)
    soup = BeautifulSoup(res.text, "html.parser")

    items = soup.select("li.Box-row")  # 1ページのリポジトリ項目

    if not items:
        print(f"[INFO] No more repositories found at page {page}.")
        break

    for item in items:
        # --- リポジトリ名 ---
        repo_tag = item.select_one("a[href*='/google/']")
        if repo_tag:
            repo_name = repo_tag.text.strip()
        else:
            continue

        owner = "google"
        repo = repo_name

        # --- URL ---
        url_repo = "https://github.com/" + repo_tag["href"]

        # --- 言語 ---
        lang_tag = item.select_one("span[itemprop='programmingLanguage']")
        language = lang_tag.text.strip() if lang_tag else None

        # --- スター数 ---
        star_tag = item.select_one("a[href$='/stargazers']")
        if star_tag:
            stars = star_tag.text.strip().replace(",", "")
            stars = int(stars) if stars.isdigit() else 0
        else:
            stars = 0

        repos.append({
            "repo_name": f"{owner}/{repo}",
            "owner": owner,
            "repo": repo,
            "language": language,
            "stars": stars,
            "url": url_repo,
            "scraped_at": datetime.now().isoformat()
        })

print(f"[INFO] Total repos scraped: {len(repos)}")


[INFO] Fetching page 1 ...
[INFO] No more repositories found at page 1.
[INFO] Total repos scraped: 0


In [11]:
# 5) 保存データの確認 (SELECT 実行して Pandas DataFrame で表示)
df = pd.read_sql_query('SELECT id, repo_name, language, stars, url, scraped_at FROM repos ORDER BY stars DESC LIMIT 200', conn)
display(df)
print(f"Total rows in DB: {len(df)}")

,id,repo_name,language,stars,url,scraped_at


Total rows in DB: 0


In [12]:
# 6) 最後に接続を閉じる
conn.close()
print('DB connection closed')

DB connection closed
